In [1]:
# build a simple rnn with cleaned test data to predict stock rice 
# 将股票涨幅看作二元分类问题
# 涨幅大于0记作1, 小于0记0
# step1: import modules
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import os
import math

path = './test_data/'
stock_code = '600000'
filename = 'cleaned'+stock_code+'.csv'
df = pd.read_csv(path+filename)
df['rise_or_not'] = df['rise'].apply(lambda x: 1 if x > 0 else 0)

In [2]:
# step2: 制定训练集和测试集
# 总数据集大小
data_sz = df.shape[0]
# 计算测试集大小, 约为原数据的%20. 向上取整
test_set_sz = math.ceil(data_sz * 0.2)
training_set_sz = data_sz - test_set_sz
# 前(data_sz - test_set_sz)天的涨幅作为训练集, 后test_set_sz天的涨幅作为测试集
training_set = df.iloc[0:training_set_sz, 1:2].values
test_set = df.iloc[-test_set_sz:, 1:2]

# 数据归一化到(0,1)之间
sc = MinMaxScaler(feature_range=(0, 1))
# 求得训练集的最大值，最小值这些训练集固有的属性，并在训练集上进行归一化
training_set_scaled = sc.fit_transform(training_set) 
test_set = sc.transform(test_set)  # 利用训练集的属性对测试集进行归一化

x_train = []
y_train = []

x_test = []
y_test = []
sample_sz = 15

# 利用for循环，遍历整个训练集，提取训练集中连续sample_sz=15天的开盘价作为输入特征x_train，
# 第16天的数据作为标签，for循环共构建training_set_sz-15组数据。
for i in range(sample_sz, len(training_set_scaled)):
    x_train.append(training_set_scaled[i - sample_sz:i, 0])
    y_train.append(training_set_scaled[i, 0])
# 对训练集进行打乱
np.random.seed(56)
np.random.shuffle(x_train)
np.random.seed(56)
np.random.shuffle(y_train)
tf.random.set_seed(56)
# 将训练集由list格式变为array格式
x_train, y_train = np.array(x_train), np.array(y_train)

for i in range(sample_sz, test_set_sz):
    x_test.append(test_set[i - sample_sz:i, 0])
    y_test.append(test_set[i, 0])
x_test, y_test = np.array(x_test), np.array(y_test)


In [3]:
model = tf.keras.models.Sequential(
    tf.keras.layers.Dense(2, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2())
)

In [4]:
model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['sparse_categorical_accuracy'])

model.fit(x_train, y_train, batch_size=16, epochs=200, validation_data=(x_test, y_test), validation_freq=1)

model.summary()

Train on 187 samples, validate on 36 samples
Epoch 1/200
187/187 [==============================] - 0s 2ms/sample - loss: 0.3008 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.1863 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/200
187/187 [==============================] - 0s 185us/sample - loss: 0.1962 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.1343 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 3/200
187/187 [==============================] - 0s 148us/sample - loss: 0.1587 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.1113 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 4/200
187/187 [==============================] - 0s 169us/sample - loss: 0.1396 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.0972 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 5/200
187/187 [==============================] - 0s 143us/sample - loss: 0.1269 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.0877 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 6/200


Epoch 44/200
187/187 [==============================] - 0s 137us/sample - loss: 0.0521 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.0245 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 45/200
187/187 [==============================] - 0s 121us/sample - loss: 0.0515 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.0240 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 46/200
187/187 [==============================] - 0s 117us/sample - loss: 0.0509 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.0235 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 47/200
187/187 [==============================] - 0s 123us/sample - loss: 0.0504 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.0230 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 48/200
187/187 [==============================] - 0s 145us/sample - loss: 0.0499 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.0226 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 49/200
187/187 [============================

187/187 [==============================] - 0s 171us/sample - loss: 0.0390 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.0133 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 88/200
187/187 [==============================] - 0s 134us/sample - loss: 0.0389 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.0131 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 89/200
187/187 [==============================] - 0s 123us/sample - loss: 0.0388 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.0130 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 90/200
187/187 [==============================] - 0s 194us/sample - loss: 0.0386 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.0129 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 91/200
187/187 [==============================] - 0s 142us/sample - loss: 0.0385 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.0127 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 92/200
187/187 [==============================] - 0s 118u

187/187 [==============================] - 0s 148us/sample - loss: 0.0355 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.0098 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 131/200
187/187 [==============================] - 0s 146us/sample - loss: 0.0354 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.0097 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 132/200
187/187 [==============================] - 0s 131us/sample - loss: 0.0354 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.0097 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 133/200
187/187 [==============================] - 0s 128us/sample - loss: 0.0354 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.0096 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 134/200
187/187 [==============================] - 0s 131us/sample - loss: 0.0353 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.0096 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 135/200
187/187 [==============================] - 0s

Epoch 173/200
187/187 [==============================] - 0s 119us/sample - loss: 0.0342 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.0083 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 174/200
187/187 [==============================] - 0s 131us/sample - loss: 0.0342 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.0083 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 175/200
187/187 [==============================] - 0s 156us/sample - loss: 0.0342 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.0083 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 176/200
187/187 [==============================] - 0s 131us/sample - loss: 0.0342 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.0082 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 177/200
187/187 [==============================] - 0s 112us/sample - loss: 0.0342 - sparse_categorical_accuracy: 0.0053 - val_loss: 0.0082 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 178/200
187/187 [======================